In [1]:
#GLOBALS
import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1) #tenth of a second delay

# print(0 if type(Arm.Arm_serial_servo_read(1)) == class(None) else int(Arm.Arm_serial_servo_read(1)))
# print(type(Arm.Arm_serial_servo_read(1)))

# print(0 if Arm.Arm_serial_servo_read(1) == None else Arm.Arm_serial_servo_read(1))
# print(Arm.Arm_serial_servo_read(1) if Arm.Arm_serial_servo_read(1) != None else 0)

#Defining variables
angle = 90
s_time = 1000 #milliseconds
gripper = 6

# Define the home configuration of the robot
H0 = np.eye(4)

print("passed init")

passed init


In [2]:
def home(): #set arm to home position after task is complete. Useful for ik
    Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-85, s_time) #FIXME
home()
print("passed home")

passed home


In [208]:
################################################################################################################################################################################################################################################################
#IK
################################################################################################################################################################################################################################################################

# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]


def rotation_matrix_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def rotation_matrix_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_matrix_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def calculate_rotation_matrix(angles):
    q1, q2, q3, q4, q5 = np.radians(angles)  # Convert degrees to radians
    R01 = rotation_matrix_z(q1)
    R12 = rotation_matrix_y(-q2)
    R23 = rotation_matrix_y(-q3)
    R34 = rotation_matrix_y(-q4)
    R45 = rotation_matrix_x(-q5)
    R0T = R01 @ R12 @ R23 @ R34 @ R45  # Matrix multiplication
    return R0T

def return_rotations(angles):
    q1, q2, q3, q4, q5 = np.radians(angles)  # Convert degrees to radians
    R01 = rotation_matrix_z(q1)
    R12 = rotation_matrix_y(-q2)
    R23 = rotation_matrix_y(-q3)
    R34 = rotation_matrix_y(-q4)
    R45 = rotation_matrix_x(-q5)
    rotations = [R01, R12, R23, R34, R45]
    return rotations

def return_pd(rotations, points):
    pd = points[0]+rotations[0] @ rotations[1] @ (points[1]+rotations[2] @ (points[2]+rotations[3] @ rotations[4] @ points[3])) 
    return pd


def main():
   
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex
    
    points = [P01, P23, P34, P5T]
    
	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]
    
    angle = 90

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
    
    qd=[angle+26, angle-90, angle-70, angle+80, angle]
    q0 = np.array(np.transpose([angle+21, angle-85, angle-75, angle+75, angle-5]))*math.pi/180
    Rd = calculate_rotation_matrix(qd)
    Pd = return_pd(return_rotations(qd), points)
    Pd_array = np.array(Pd)
    Pd = Pd_array.reshape(3, 1)
    
    # compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time) #FIXME
    Arm.Arm_serial_servo_write6(q[0], q[1], q[2], q[3], q[4], 90, s_time) #FIXME
    
if __name__ == "__main__" :
    main()
    
print("passed IK")

[115.76498157   0.29039628  19.64124541 169.81042313  89.78680669]
passed IK


In [206]:
################################################################################################################################################################################################################################################################
#FEEDBACK TASK
################################################################################################################################################################################################################################################################
class PIDController:
    def __init__(self, P, I, D):
        self.Kp = P
        self.Ki = I
        self.Kd = D
        self.previous_error = 0
        self.integral = 0

    def update(self, error, dt):
        # Proportional term
        P_term = self.Kp * error
        # Integral term
        self.integral += error * dt
        I_term = self.Ki * self.integral
        # Derivative term
        derivative = (error - self.previous_error) / dt
        D_term = self.Kd * derivative
        # Update previous error
        self.previous_error = error
        # Total output
        return P_term + I_term + D_term

In [207]:
################################################################################################################################################################################################################################################################
#PATH FOLLOWING TASK
################################################################################################################################################################################################################################################################
#IMPLEMENTED FUNCTIONS
# import numpy as np
# Other imports remain the same as your provided code

# Assuming Arm_Lib is the module that communicates with your robotic arm
from Arm_Lib import Arm_Device
import numpy as np
import general_robotics_toolbox as rox #ik
import time
import math
from Arm_Lib import Arm_Device #

class PIDController:
    def __init__(self, P, I, D):
        self.Kp = P
        self.Ki = I
        self.Kd = D
        self.previous_error = 0
        self.integral = 0

    def update(self, error, dt):
        # Proportional term
        P_term = self.Kp * error
        # Integral term
        self.integral += error * dt
        I_term = self.Ki * self.integral
        # Derivative term
        derivative = (error - self.previous_error) / dt
        D_term = self.Kd * derivative
        # Update previous error
        self.previous_error = error
        # Total output
        return P_term + I_term + D_term

# Step 1: Generate a joint space path
def generate_joint_space_path(start_angles, end_angles, steps):
    path = [np.linspace(start, end, steps) for start, end in zip(start_angles, end_angles)]
    return np.transpose(path)

# Step 2: Command the arm to follow the path
def command_arm_to_follow_path(robot, path, s_time):
    for joint_angles in path:
        # Convert angles from radians to degrees for the robot's actuators
        angles_in_degrees = joint_angles * (180 / np.pi)
        # Assuming Arm_Device is instantiated and the function Arm_serial_servo_write6 is available
        # to send the joint angles to the robot. Replace 'Arm_Device' with your actual object.
        Arm.Arm_serial_servo_write6(angles_in_degrees[0], angles_in_degrees[1], angles_in_degrees[2], angles_in_degrees[3], angles_in_degrees[4], 90, 1500)
        time.sleep(0.1)  # Wait for some time interval before sending the next set of angles
        
def command_arm_to_follow_path_with_feedback(robot, path, s_time, pid_controllers):
    for joint_angles in path:
        actual_joint_angles = measure_actual_position(robot)
        error = joint_angles - actual_joint_angles
        adjusted_angles = []

        for i in range(len(joint_angles)):
            dt = 0.1  # Assuming 0.1 seconds as the time interval
            adjustment = pid_controllers[i].update(error[i], dt)
            adjusted_angles.append(joint_angles[i] + adjustment)

        angles_in_degrees = np.array(adjusted_angles) * (180 / np.pi)
        Arm.Arm_serial_servo_write6(*angles_in_degrees, 90, 1500)
        time.sleep(dt)

# Step 3: Measure the actual position (this function should be replaced with actual sensor readout)
def measure_actual_position(robot):
    joint_angles = []
    for i in range(1, 6):
        if Arm.Arm_serial_servo_read(i) != None:
            time.sleep(0.1)
            joint_angles.append(Arm.Arm_serial_servo_read(i))
        else:
            time.sleep(0.1)
            joint_angles.append(0)
    print(joint_angles)
    # Convert the servo positions to angles, if necessary
    # joint_angles = [convert_position_to_angle(pos) for pos in joint_angles]
    time.sleep(0.1)
    return np.array(joint_angles)

# Step 4: Evaluate the error
def evaluate_path_following_error(robot, desired_path):
    error_path = []
    for desired_joint_angles in desired_path:
        actual_joint_angles = measure_actual_position(robot)
        error = np.array(desired_joint_angles) - np.array(actual_joint_angles)
        error_path.append(error)
    return error_path

def main_path():
 
    s_time = 1500 #milliseconds
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])
    #qd=[0;45;135;45;135]
    Rd = np.array([ [0.43171, 0.076122, -0.89879], [-0.88513, -0.15607, -0.43837], [-0.17365, 0.98481, 0]])
    Pd = np.array([[-0.0303252], [-0.0537649], [0.0951899]])
	# convert initial guess to radians
    q0 = np.array(np.transpose([90, 90, 90, 90, 90]))*math.pi/180

    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3
    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ez
    P34 = l3 * ez
    P45 = np.zeros(3)
    P5T = (l4 + l5) * ez

    H = np.array([ez, ez, ez, ez, ez]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]
    robot = rox.Robot(H, P, joint_type)
    # Define your start and end angles here (in radians)
    start_angles = np.array([90, 90, 90, 90, 90]) * np.pi / 180
    end_angles = np.array([45, 150, -60, 95, 90]) * np.pi / 180
    steps = 5  # Define the number of steps you want in the path

    pid_controllers = [PIDController(0.0000001, 0.000000001, 0.000000001) for _ in range(5)]
    
    # Generate the path
    joint_space_path = generate_joint_space_path(start_angles, end_angles, steps)
    
    # Command the arm to follow the path
#     command_arm_to_follow_path(robot, joint_space_path, s_time)

    command_arm_to_follow_path_with_feedback(robot, joint_space_path, s_time, pid_controllers)
    
    # Measure the error
    path_following_error = evaluate_path_following_error(robot, joint_space_path)

    # Print or process the error as needed
    print("Path following errors:")
    print(path_following_error)

if __name__ == "__main__":
    main_path()
    
print("passed Path Following")

[116, 0, 20, 170, 90]
[113, 14, 36, 145, 89]
[100, 56, 55, 117, 90]
[87, 86, 45, 103, 90]
[75, 108, 20, 98, 89]
[63, 126, 0, 96, 89]
[54, 140, 0, 96, 90]
[45, 150, 0, 96, 90]
[45, 150, 0, 96, 90]
[45, 150, 0, 96, 89]
Path following errors:
[array([ -61.42920367, -124.42920367,    1.57079633,  -94.42920367,
        -87.42920367]), array([ -52.62555321, -138.16740429,    0.91629786,  -94.40738706,
        -88.42920367]), array([ -43.82190275, -147.9056049 ,    0.26179939,  -94.38557044,
        -88.42920367]), array([ -44.0182523 , -147.64380551,   -0.39269908,  -94.36375383,
        -88.42920367]), array([ -44.21460184, -147.38200612,   -1.04719755,  -94.34193721,
        -87.42920367])]
passed Path Following


In [184]:
################################################################################################################################################################################################################################################################
#FK Function 1
################################################################################################################################################################################################################################################################
s_time = 1500 #milliseconds
home()
time.sleep(2) 
#from home position to blue square where cups are stacked, 35 cups
def home_to_cupstack():
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+50, angle, angle, s_time)
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle, s_time)
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 
    time.sleep(1)
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 

home_to_cupstack()

In [168]:
################################################################################################################################################################################################################################################################
#FK Function 2
################################################################################################################################################################################################################################################################

s_time = 1500 #milliseconds

def cupstack_to_dispenser():
    Arm.Arm_serial_servo_write6(angle-45, angle+60, angle-150, angle+5, angle, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-45, angle+45, angle-150, angle+45, angle, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle+45, angle-150, angle+45, angle+180, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle-90, angle-70, angle+80, angle+180, angle+57, s_time) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle-26, angle-90, angle-70, angle+80, angle+180, angle, 25) 
    time.sleep(1.5)
    Arm.Arm_serial_servo_write6(angle+26, angle-90, angle-70, angle+80, angle+180, angle, s_time) 

cupstack_to_dispenser()

In [ ]:
def main():
#     home() #add inherit delay to function 

try :
    main()
except KeyboardInterrupt:
     print(" Program closed! ")
     pass